# International data - World Cup 2018 predictions

Ported from Excel - see original [here](models/World cup 2018 CALC.xlsx)

### 1. Input data on team fixtures and performance

In [1]:
import pandas as pd

In [2]:
HOME_TEAMS = ["Russia"]
HOME_TEAMS

['Russia']

In [3]:
fixtures = pd.read_csv("../data/raw/whs/whs_fix/whs_fix_wcm_2018.csv")
fixtures.columns = ["Date", "Time", "ignore_1", "Team1", "ignore_2", "Team2", "ignore_3"]
fixtures.drop(columns=["ignore_1", "ignore_2", "ignore_3"], inplace=True)
fixtures.Date.fillna(method="ffill", inplace=True)
fixtures.dropna(axis="index", subset=["Team1"], inplace=True)
fixtures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 1 to 88
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    64 non-null     object
 1   Time    64 non-null     object
 2   Team1   64 non-null     object
 3   Team2   64 non-null     object
dtypes: object(4)
memory usage: 2.5+ KB


In [4]:
fixtures["HomeAdv1"] = 0
fixtures["HomeAdv2"] = 0
fixtures.loc[fixtures.Team1.isin(HOME_TEAMS), "HomeAdv1"] = 1
fixtures.loc[fixtures.Team2.isin(HOME_TEAMS), "HomeAdv2"] = 1

fixtures.head(5)

,Date,Time,Team1,Team2,HomeAdv1,HomeAdv2
1,"Thursday, Jun 14 2018",16:00,Russia,Saudi Arabia,1,0
3,"Friday, Jun 15 2018",13:00,Egypt,Uruguay,0,0
4,"Friday, Jun 15 2018",16:00,Morocco,Iran,0,0
5,"Friday, Jun 15 2018",19:00,Portugal,Spain,0,0
7,"Saturday, Jun 16 2018",11:00,France,Australia,0,0


In [5]:
elo = pd.read_csv("../data/raw/wkp/wkp_elo/World_Football_Elo_Ratings.csv")
elo.columns = ["Team", "EloRank", "ignore_1", "ignore_2", "ignore_3", "EloRating", "FIFARank", "ignore_4", "ignore_5"]
elo.drop(columns=["ignore_1", "ignore_2", "ignore_3", "ignore_4", "ignore_5"], inplace=True)
elo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Team       100 non-null    object
 1   EloRank    100 non-null    int64 
 2   EloRating  100 non-null    int64 
 3   FIFARank   100 non-null    object
dtypes: int64(2), object(2)
memory usage: 3.2+ KB


In [6]:
elo.head(5)

,Team,EloRank,EloRating,FIFARank
0,Brazil,1,2131,2
1,Germany,2,2092,1
2,Spain,3,2049,8
3,France,4,1987,7
4,Argentina,5,1985,5


In [7]:
qualifying = pd.read_csv("../data/raw/fif/Qualifying_goals.csv")
qualifying = qualifying[["Team", "Rank"]]
qualifying.columns = ["Team", "QualifyGoalsRank"]
qualifying.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Team              32 non-null     object
 1   QualifyGoalsRank  32 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 640.0+ bytes


In [8]:
qualifying.head(5)

,Team,QualifyGoalsRank
0,Russia,16
1,Saudi Arabia,6
2,Egypt,26
3,Uruguay,11
4,Portugal,5


In [9]:
data = fixtures.merge(elo, how="inner", left_on="Team1", right_on="Team", suffixes=["","1"])\
    .drop(columns=["Team"])\
    .merge(elo, how="inner", left_on="Team2", right_on="Team", suffixes=["1","2"])\
    .merge(qualifying, how="inner", left_on="Team1", right_on="Team")\
    .drop(columns=["Team_x", "Team_y"])\
    .merge(qualifying, how="inner", left_on="Team2", right_on="Team")\
    .drop(columns=["Team"])\
    .rename(columns={"QualifyGoalsRank_x": "QualifyGoalsRank1", "QualifyGoalsRank_y": "QualifyGoalsRank2"})
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 0 to 63
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Date               64 non-null     object
 1   Time               64 non-null     object
 2   Team1              64 non-null     object
 3   Team2              64 non-null     object
 4   HomeAdv1           64 non-null     int64 
 5   HomeAdv2           64 non-null     int64 
 6   EloRank1           64 non-null     int64 
 7   EloRating1         64 non-null     int64 
 8   FIFARank1          64 non-null     object
 9   EloRank2           64 non-null     int64 
 10  EloRating2         64 non-null     int64 
 11  FIFARank2          64 non-null     object
 12  QualifyGoalsRank1  64 non-null     int64 
 13  QualifyGoalsRank2  64 non-null     int64 
dtypes: int64(8), object(6)
memory usage: 7.5+ KB


### 2. Build model and make predictions

In [10]:
GOAL_WEIGHT = 4.
GOAL_BOOST = 19.
GOAL_WEIGHT, GOAL_BOOST

(4.0, 19.0)

In [43]:
data["EloRatingDiff"] = data["EloRating1"] - data["EloRating2"]
data["EloRatingDiffWithHomeAdv"] = data["EloRatingDiff"] + (100 * data.HomeAdv1) - (100 * data.HomeAdv2)
data["WinExpectency1Square"] = (10**((-data.EloRatingDiffWithHomeAdv)/400))+1
data["WinExpectency1"] = data["WinExpectency1Square"]**-1
data["RawGoalDiff"] = (GOAL_WEIGHT * (data.WinExpectency1 - 0.5)).round(0)
data["RawGoalDiffAbs"] = data["RawGoalDiff"].abs()
data["EitherWins"] = 0
data.loc[data.RawGoalDiffAbs > 0, "EitherWins"] = 1
data["QualifyGoalsRankAvg"] = (data["QualifyGoalsRank1"] + data["QualifyGoalsRank2"]) / 2
data["ApplyGoalBoost"] = 0
data.loc[data.QualifyGoalsRankAvg <= GOAL_BOOST, "ApplyGoalBoost"] = 1
data["Goals1"] = data["ApplyGoalBoost"]
data.loc[data.RawGoalDiff > 0, "Goals1"] = data.RawGoalDiff + data.ApplyGoalBoost
data["Goals2"] = data["ApplyGoalBoost"]
data.loc[data.RawGoalDiff <= 0, "Goals2"] = data.ApplyGoalBoost - data.RawGoalDiff
data["GoalDiff"] = data.Goals1 - data.Goals2
data["GoalDiffAbs"] = data.GoalDiff.abs()
data["GoalTotal"] = data.Goals1 + data.Goals2
data.iloc[:, -14:].head(5)

,EloRatingDiff,EloRatingDiffWithHomeAdv,WinExpectency1Square,WinExpectency1,RawGoalDiff,RawGoalDiffAbs,EitherWins,QualifyGoalsRankAvg,ApplyGoalBoost,Goals1,Goals2,GoalDiff,GoalDiffAbs,GoalTotal
0,88,188,1.338844,0.746913,1.0,1.0,1,11.0,1,2.0,1.0,1.0,1.0,3.0
1,294,294,1.184077,0.844540,1.0,1.0,1,8.5,1,2.0,1.0,1.0,1.0,3.0
2,42,142,1.441570,0.693688,1.0,1.0,1,21.0,0,1.0,0.0,1.0,1.0,1.0
3,-46,-46,2.303167,0.434185,-0.0,0.0,0,16.0,1,1.0,1.0,0.0,0.0,2.0
4,-168,-68,2.479108,0.403371,-0.0,0.0,0,22.0,0,0.0,0.0,0.0,0.0,0.0


In [44]:
data.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Date,64,25,"Saturday, Jun 16 2018",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Time,64,8,19:00,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Team1,64,32,France,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Team2,64,32,England,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HomeAdv1,64,NaN,NaN,NaN,0.046875,0.213042,0,0,0,0,1
HomeAdv2,64,NaN,NaN,NaN,0.03125,0.175368,0,0,0,0,1
EloRank1,64,NaN,NaN,NaN,19.1562,15.9187,1,6,16.5,27,63
EloRating1,64,NaN,NaN,NaN,1864.58,137.747,1597,1751,1855,1967,2131
FIFARank1,64,32,7,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EloRank2,64,NaN,NaN,NaN,22.5625,16.9742,1,8,17,40,63


### 3. Evaluate predictions against historic trends

In [45]:
poisson = pd.read_csv("../data/raw/wkp/wkp_pds/Poisson_goals_World_Cup.csv")
poisson.columns = ["k", "P_of_k_goals", "ignore_1", "ignore_2"]
poisson.drop(columns=["ignore_1", "ignore_2"], inplace=True)
poisson["label"] = "% games "+poisson.k.astype(str)+" goals"
poisson.set_index("label", inplace=True)
poisson.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, % games 0 goals to % games 7 goals
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   k             8 non-null      int64  
 1   P_of_k_goals  8 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 192.0+ bytes


In [46]:
poisson

,k,P_of_k_goals
label,,
% games 0 goals,0,0.082
% games 1 goals,1,0.205
% games 2 goals,2,0.257
% games 3 goals,3,0.213
% games 4 goals,4,0.133
% games 5 goals,5,0.067
% games 6 goals,6,0.028
% games 7 goals,7,0.010


In [47]:
totals = pd.read_csv("../data/raw/fif/World_Cup_goals.csv")
totals.columns = ["Tournament", "NoOfTeams", "MatchesPlayed", "GoalsScored", "AverageGoals", "AverageAttendance",
                 "Rolling", "ignore_1", "ignore_2"]
totals.drop(columns=["ignore_1", "ignore_2"], inplace=True)
totals.dropna(axis="index", subset=["Tournament"], inplace=True)
totals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tournament         20 non-null     object 
 1   NoOfTeams          20 non-null     float64
 2   MatchesPlayed      20 non-null     float64
 3   GoalsScored        20 non-null     float64
 4   AverageGoals       20 non-null     float64
 5   AverageAttendance  20 non-null     object 
 6   Rolling            20 non-null     float64
dtypes: float64(5), object(2)
memory usage: 1.2+ KB


In [48]:
totals

,Tournament,NoOfTeams,MatchesPlayed,GoalsScored,AverageGoals,AverageAttendance,Rolling
0,2014 FIFA World Cup Brazil ™,32.0,64.0,171.0,2.7,"52,918",2.7
1,2010 FIFA World Cup South Africa ™,32.0,64.0,145.0,2.3,"49,669",2.5
2,2006 FIFA World Cup Germany ™,32.0,64.0,147.0,2.3,"52,491",2.4
3,2002 FIFA World Cup Korea/Japan ™,32.0,64.0,161.0,2.5,"42,268",2.4
4,1998 FIFA World Cup France ™,32.0,64.0,171.0,2.7,"43,517",2.5
5,1994 FIFA World Cup USA ™,24.0,52.0,141.0,2.7,"68,991",2.5
6,1990 FIFA World Cup Italy ™,24.0,52.0,115.0,2.2,"48,388",2.5
7,1986 FIFA World Cup Mexico ™,24.0,52.0,132.0,2.5,"46,039",2.5
8,1982 FIFA World Cup Spain ™,24.0,52.0,146.0,2.8,"40,571",2.5
9,1978 FIFA World Cup Argentina ™,16.0,38.0,102.0,2.7,"40,678",2.5


In [49]:
historic = pd.concat([poisson.P_of_k_goals.T, totals.iloc[10, -1:]])
historic.rename({"Rolling": "AverageGoalsPerGame"}, inplace=True)
historic["% games drawn"] = 314 / 1416
historic["% games won"] = 1 - historic["% games drawn"]
historic

% games 0 goals           0.082
% games 1 goals           0.205
% games 2 goals           0.257
% games 3 goals           0.213
% games 4 goals           0.133
% games 5 goals           0.067
% games 6 goals           0.028
% games 7 goals            0.01
AverageGoalsPerGame         2.5
% games drawn          0.221751
% games won            0.778249
dtype: object

In [50]:
predictions = pd.DataFrame([{
    "AverageGoalsPerGame": data.GoalTotal.sum() / data.shape[0],
    "% games drawn": data[data.EitherWins == 0].shape[0] / data.shape[0],
    "% games won": data[data.EitherWins == 1].shape[0] / data.shape[0],
}])
for g in range(8):
    predictions["% games "+str(g)+" goals"] = data[data.GoalTotal == g].shape[0] / data.shape[0]
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AverageGoalsPerGame  1 non-null      float64
 1   % games drawn        1 non-null      float64
 2   % games won          1 non-null      float64
 3   % games 0 goals      1 non-null      float64
 4   % games 1 goals      1 non-null      float64
 5   % games 2 goals      1 non-null      float64
 6   % games 3 goals      1 non-null      float64
 7   % games 4 goals      1 non-null      float64
 8   % games 5 goals      1 non-null      float64
 9   % games 6 goals      1 non-null      float64
 10  % games 7 goals      1 non-null      float64
dtypes: float64(11)
memory usage: 216.0 bytes


In [51]:
hist_vs_pred = pd.concat([pd.DataFrame(historic).T, predictions]).T
hist_vs_pred.columns = ["Historic", "Predictions"]
hist_vs_pred["%Diff"] = 1.0
hist_vs_pred.loc[hist_vs_pred.Predictions > 0, "%Diff"] = 1 - (hist_vs_pred.loc[hist_vs_pred.Predictions > 0, "Historic"]\
                                                               / hist_vs_pred.loc[hist_vs_pred.Predictions > 0, "Predictions"])
hist_vs_pred

,Historic,Predictions,%Diff
% games 0 goals,0.082,0.125,0.344
% games 1 goals,0.205,0.171875,-0.192727
% games 2 goals,0.257,0.21875,-0.174857
% games 3 goals,0.213,0.4375,0.513143
% games 4 goals,0.133,0.046875,-1.83733
% games 5 goals,0.067,0,1
% games 6 goals,0.028,0,1
% games 7 goals,0.01,0,1
AverageGoalsPerGame,2.5,2.10938,-0.185185
% games drawn,0.221751,0.34375,0.354905


### 4. Compare predictions to actual results

In [52]:
results = pd.read_excel("../models/World cup 2018 CALC.xlsx", sheet_name="Fixtures_and_calcs")
results = results.iloc[:, [0,1,3,5,32,33]]
results.columns = ["Date", "Time", "Team1", "Team2", "Actual1", "Actual2"]
results.Date.fillna(method="ffill", inplace=True)
results["Time"] = results["Time"].astype(str).str[:5]
results.dropna(axis="index", subset=["Team1"], inplace=True)
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 1 to 88
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     64 non-null     object 
 1   Time     64 non-null     object 
 2   Team1    64 non-null     object 
 3   Team2    64 non-null     object 
 4   Actual1  64 non-null     float64
 5   Actual2  64 non-null     float64
dtypes: float64(2), object(4)
memory usage: 3.5+ KB


In [53]:
results.head(5)

,Date,Time,Team1,Team2,Actual1,Actual2
1,"Thursday, Jun 14 2018",16:00,Russia,Saudi Arabia,5.0,0.0
3,"Friday, Jun 15 2018",13:00,Egypt,Uruguay,0.0,1.0
4,"Friday, Jun 15 2018",16:00,Morocco,Iran,0.0,1.0
5,"Friday, Jun 15 2018",19:00,Portugal,Spain,3.0,3.0
7,"Saturday, Jun 16 2018",11:00,France,Australia,2.0,1.0


In [54]:
compare = data[["Date", "Time", "Team1", "Team2", "Goals1", "Goals2"]]\
    .merge(results, how="inner", on=["Date", "Time", "Team1", "Team2"])
compare.rename(columns={"Goals1": "Prediction1", "Goals2": "Prediction2"}, inplace=True)

compare["Prediction2"] = compare.Prediction2.astype(float)

compare["CorrectResult"] = 0
compare.loc[((compare.Prediction1 > compare.Prediction2) & (compare.Actual1 > compare.Actual2))
            | ((compare.Prediction1 == compare.Prediction2) & (compare.Actual1 == compare.Actual2))
            | ((compare.Prediction1 < compare.Prediction2) & (compare.Actual1 < compare.Actual2))
            , "CorrectResult"] = 1

compare["CorrectGoalDiff"] = 0
compare.loc[(compare.Prediction1 - compare.Prediction2 == compare.Actual1 - compare.Actual2), "CorrectGoalDiff"] = 1

compare["CorrectScore"] = 0
compare.loc[(compare.Prediction1 == compare.Actual1) & (compare.Prediction2 == compare.Actual2), "CorrectScore"] = 1

compare["PointsScored"] = compare.CorrectResult + compare.CorrectGoalDiff + compare.CorrectScore

compare.tail(10)

,Date,Time,Team1,Team2,Prediction1,Prediction2,Actual1,Actual2,CorrectResult,CorrectGoalDiff,CorrectScore,PointsScored
54,"Sunday, Jun 17 2018",13:00,Costa Rica,Serbia,1.0,1.0,0.0,1.0,0,0,0,0
55,"Saturday, Jun 23 2018",19:00,Germany,Sweden,2.0,1.0,2.0,1.0,1,1,1,3
56,"Wednesday, Jun 27 2018",15:00,Mexico,Sweden,1.0,1.0,0.0,3.0,0,0,0,0
57,"Wednesday, Jun 27 2018",15:00,South Korea,Germany,1.0,3.0,2.0,0.0,0,0,0,0
58,"Wednesday, Jun 27 2018",19:00,Serbia,Brazil,1.0,3.0,0.0,2.0,1,1,0,2
59,"Tuesday, Jun 19 2018",16:00,Poland,Senegal,1.0,1.0,1.0,2.0,0,0,0,0
60,"Sunday, Jun 24 2018",16:00,Japan,Senegal,1.0,1.0,2.0,2.0,1,1,0,2
61,"Sunday, Jun 24 2018",19:00,Poland,Colombia,1.0,2.0,0.0,3.0,1,0,0,1
62,"Thursday, Jun 28 2018",15:00,Senegal,Colombia,0.0,1.0,0.0,1.0,1,1,1,3
63,"Thursday, Jun 28 2018",15:00,Japan,Poland,1.0,2.0,0.0,1.0,1,1,0,2


In [55]:
compare[compare.PointsScored == 0]

,Date,Time,Team1,Team2,Prediction1,Prediction2,Actual1,Actual2,CorrectResult,CorrectGoalDiff,CorrectScore,PointsScored
3,"Monday, Jun 25 2018",15:00,Saudi Arabia,Egypt,1.0,1.0,2.0,1.0,0,0,0,0
6,"Thursday, Jun 21 2018",19:00,Argentina,Croatia,1.0,0.0,0.0,3.0,0,0,0,0
7,"Tuesday, Jun 26 2018",19:00,Iceland,Croatia,0.0,0.0,1.0,2.0,0,0,0,0
9,"Sunday, Jul 1 2018",15:00,Spain,Russia,2.0,1.0,1.0,1.0,0,0,0,0
10,"Saturday, Jun 30 2018",19:00,Uruguay,Portugal,1.0,1.0,2.0,1.0,0,0,0,0
11,"Monday, Jun 25 2018",19:00,Iran,Portugal,1.0,2.0,1.0,1.0,0,0,0,0
13,"Tuesday, Jun 26 2018",15:00,Denmark,France,1.0,2.0,0.0,0.0,0,0,0,0
15,"Thursday, Jun 21 2018",13:00,Denmark,Australia,2.0,1.0,1.0,1.0,0,0,0,0
16,"Thursday, Jun 21 2018",16:00,France,Peru,1.0,1.0,1.0,0.0,0,0,0,0
18,"Saturday, Jun 30 2018",15:00,France,Argentina,0.0,0.0,4.0,3.0,0,0,0,0


In [56]:
compare[["CorrectResult", "CorrectGoalDiff", "CorrectScore", "PointsScored"]].sum()

CorrectResult      34
CorrectGoalDiff    21
CorrectScore        5
PointsScored       60
dtype: int64

### 5. Improve the model

## TODO

(parity with Excel)
* Compare predictions to actual

(enhancements)
* Turn model into class/function
* Tune hyperparameters
* Output predictions